 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates


 ## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [2]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    log_level="DEBUG",
)


 ## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [3]:
coordinates = AustralianCoordinates(latitude=-33.86, longitude=151.21)  # Sydney, NSW
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata


[10/23/25 20:05:29] DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://api.met.no/weatherapi/locationforecast/2.0/compact

[10/23/25 20:05:30] DEBUG    Cached response for: 4a5961d27789c123038238e4def09a11

                    INFO     Request successful on attempt 1

{'updated_at': '2025-10-23T11:17:22Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

In [4]:
response

MetNoResponse(raw_data={'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [151.21, -33.86, 24]}, 'properties': {'meta': {'updated_at': '2025-10-23T11:17:22Z', 'units': {'air_pressure_at_sea_level': 'hPa', 'air_temperature': 'celsius', 'cloud_area_fraction': '%', 'precipitation_amount': 'mm', 'relative_humidity': '%', 'wind_from_direction': 'degrees', 'wind_speed': 'm/s'}}, 'timeseries': [{'time': '2025-10-23T11:00:00Z', 'data': {'instant': {'details': {'air_pressure_at_sea_level': 1015.4, 'air_temperature': 18.4, 'cloud_area_fraction': 89.8, 'relative_humidity': 62.3, 'wind_from_direction': 43.0, 'wind_speed': 5.5}}, 'next_12_hours': {'summary': {'symbol_code': 'fair_day'}, 'details': {}}, 'next_1_hours': {'summary': {'symbol_code': 'cloudy'}, 'details': {'precipitation_amount': 0.0}}, 'next_6_hours': {'summary': {'symbol_code': 'partlycloudy_night'}, 'details': {'precipitation_amount': 0.0}}}}, {'time': '2025-10-23T12:00:00Z', 'data': {'instant': {'details': {'air_pressu

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload into the daily

 schema (`DailyWeatherSummary`) or expose the raw hourly data by disabling the

 aggregation flag.

In [5]:
daily_forecasts = api.to_dataframe(response, aggregate_to_daily=True)
hourly_forecasts = api.to_dataframe(response, aggregate_to_daily=False)

daily_forecasts.head()


,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.1,21.2,NaN,3.953846,5.5,65.938462,1013.646154,41.938462,cloudy
1,2025-10-24,15.1,23.9,0.1,5.554167,11.1,71.045833,1015.545833,69.983333,lightrain
2,2025-10-25,15.8,22.1,NaN,5.662500,9.9,73.633333,1011.554167,63.379167,partlycloudy_day
3,2025-10-26,18.7,26.9,0.4,2.950000,4.1,52.275000,1001.600000,44.925000,cloudy
4,2025-10-27,14.8,24.9,1.7,7.650000,10.8,54.300000,1010.400000,66.025000,rainshowers_day


In [6]:
hourly_forecasts.head()


,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,next_1_hours_precipitation_amount,next_6_hours_precipitation_amount
0,2025-10-23 11:00:00+00:00,1015.4,18.4,89.8,62.3,43.0,5.5,0.0,NaN
1,2025-10-23 12:00:00+00:00,1014.9,18.4,56.2,61.8,34.5,5.5,0.0,NaN
2,2025-10-23 13:00:00+00:00,1015.0,18.4,99.2,62.4,17.5,4.2,0.0,NaN
3,2025-10-23 14:00:00+00:00,1014.1,18.3,33.6,64.3,10.0,3.8,0.0,NaN
4,2025-10-23 15:00:00+00:00,1013.4,17.4,13.3,68.9,325.2,3.1,0.0,NaN


 ### Quick helper: `get_daily_forecast`

 Prefer `get_daily_forecast` if you simply need the daily summaries as Pydantic

 models and want to control the number of days returned.

In [7]:
summaries = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

pd.DataFrame([summary.model_dump() for summary in summaries])


                    DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-33.86&lon=151.21

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Cache hit for request: 4a5961d27789c123038238e4def09a11

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-23,16.1,21.2,NaN,3.953846,5.5,65.938462,1013.646154,41.938462,cloudy
1,2025-10-24,15.1,23.9,0.1,5.554167,11.1,71.045833,1015.545833,69.983333,lightrain
2,2025-10-25,15.8,22.1,NaN,5.662500,9.9,73.633333,1011.554167,63.379167,partlycloudy_day
3,2025-10-26,18.7,26.9,0.4,2.950000,4.1,52.275000,1001.600000,44.925000,cloudy
4,2025-10-27,14.8,24.9,1.7,7.650000,10.8,54.300000,1010.400000,66.025000,rainshowers_day


 ## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [8]:
first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = history_end - pd.Timedelta(days=4)

silo_api = SiloAPI(log_level="INFO")
raw_silo = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=history_end.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp"],
)

silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

silo_history["date"] = pd.to_datetime(silo_history["date"])

required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
missing = [col for col in required_cols if col not in silo_history.columns]
if missing:
    raise ValueError(f"Missing required columns from SILO response: {missing}")

silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)

merged


[10/23/25 20:05:31] INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 5 SILO records with 11 met.no records. Total: 16 records

,avg_wind_speed,avg_cloud_fraction,forecast_generated_at,dominant_weather_symbol,max_temp,min_temp,vp,max_wind_speed,date,mslp,daily_rain,is_forecast,data_source
0,NaN,NaN,NaT,NaN,22.0,15.7,NaN,NaN,2025-10-18,NaN,0.7,False,silo
1,NaN,NaN,NaT,NaN,25.5,15.5,NaN,NaN,2025-10-19,NaN,0.0,False,silo
2,NaN,NaN,NaT,NaN,36.0,17.0,NaN,NaN,2025-10-20,NaN,0.0,False,silo
3,NaN,NaN,NaT,NaN,23.7,17.7,NaN,NaN,2025-10-21,NaN,0.0,False,silo
4,NaN,NaN,NaT,NaN,22.8,18.7,NaN,NaN,2025-10-22,NaN,0.0,False,silo
5,3.953846,41.938462,2025-10-23 12:05:31.112318+00:00,cloudy,21.2,16.1,65.938462,5.5,2025-10-23,1013.646154,NaN,True,metno
6,5.554167,69.983333,2025-10-23 12:05:31.112318+00:00,lightrain,23.9,15.1,71.045833,11.1,2025-10-24,1015.545833,0.1,True,metno
7,5.662500,63.379167,2025-10-23 12:05:31.112318+00:00,partlycloudy_day,22.1,15.8,73.633333,9.9,2025-10-25,1011.554167,NaN,True,metno
8,2.950000,44.925000,2025-10-23 12:05:31.112318+00:00,cloudy,26.9,18.7,52.275000,4.1,2025-10-26,1001.600000,0.4,True,metno
9,7.650000,66.025000,2025-10-23 12:05:31.112318+00:00,rainshowers_day,24.9,14.8,54.300000,10.8,2025-10-27,1010.400000,1.7,True,metno


In [10]:
merge_summary = get_merge_summary(merged)
merge_summary


{'total_records': 16,
 'silo_records': np.int64(5),
 'metno_records': np.int64(11),
 'date_range': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-11-02 00:00:00'),
  'days': 16},
 'silo_period': {'start': Timestamp('2025-10-18 00:00:00'),
  'end': Timestamp('2025-10-22 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-23 00:00:00'),
  'end': Timestamp('2025-11-02 00:00:00')},
 'transition_date': Timestamp('2025-10-23 00:00:00')}

## 4b. Merge Forecast with SILO PatchedPoint Data

In [ ]:

first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date #- pd.Timedelta(days=1) # Include day before forecast to check overlap handling
history_start = history_end - pd.Timedelta(days=8)

silo_api = SiloAPI(log_level="INFO")
raw_silo = silo_api.search_stations("Northam")

# {'station_code': 10111,
#  'name': 'NORTHAM',
#  'latitude': -31.651,
#  'longitude': 116.659,
#  'state': 'WA',
#  'elevation': 170.0}

silo_station_data = silo_api.get_station_data(station_code="10111",
						  start_date=history_start.strftime("%Y%m%d"),
						end_date=history_end.strftime("%Y%m%d"),
						variables=["rainfall", "max_temp", "min_temp", "evaporation"],
		)

silo_station_data = silo_station_data.rename(columns={"YYYY-MM-DD": "date"}).copy()


merge_historical_and_forecast(
    silo_data=silo_station_data,
    metno_data=daily_forecasts,
    fill_missing=True,
    overlap_strategy="prefer_silo",
)



                    INFO     Request successful on attempt 1

                    INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-23 00:00:00

/Users/hfsi/Developer/weather_tools/src/weather_tools/merge_weather_data.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([silo_df, metno_df], ignore_index=True)


                    INFO     Merged 8 SILO records with 11 met.no records. Total: 19 records

,avg_wind_speed,station,max_temp_source,avg_cloud_fraction,evap_pan_source,daily_rain_source,forecast_generated_at,min_temp_source,dominant_weather_symbol,max_temp,min_temp,vp,max_wind_speed,evap_pan,date,mslp,daily_rain,is_forecast,data_source
0,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,24.7,10.6,NaN,NaN,4.6,2025-10-15,NaN,0.0,False,silo
1,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,27.7,8.0,NaN,NaN,6.7,2025-10-16,NaN,0.0,False,silo
2,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,34.0,9.1,NaN,NaN,6.8,2025-10-17,NaN,0.0,False,silo
3,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,24.4,15.9,NaN,NaN,6.9,2025-10-18,NaN,0.0,False,silo
4,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,22.9,7.5,NaN,NaN,4.9,2025-10-19,NaN,0.0,False,silo
5,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,21.4,6.0,NaN,NaN,3.2,2025-10-20,NaN,0.0,False,silo
6,NaN,10111.0,0.0,NaN,25.0,0.0,NaT,0.0,NaN,22.5,9.0,NaN,NaN,4.5,2025-10-21,NaN,19.8,False,silo
7,NaN,10111.0,75.0,NaN,0.0,0.0,NaT,0.0,NaN,25.5,8.7,NaN,NaN,0.0,2025-10-22,NaN,0.0,False,silo
8,3.953846,NaN,NaN,41.938462,NaN,NaN,2025-10-23 12:05:31.417935+00:00,NaN,cloudy,21.2,16.1,65.938462,5.5,NaN,2025-10-23,1013.646154,NaN,True,metno
9,5.554167,NaN,NaN,69.983333,NaN,NaN,2025-10-23 12:05:31.417935+00:00,NaN,lightrain,23.9,15.1,71.045833,11.1,NaN,2025-10-24,1015.545833,0.1,True,metno


In [12]:
daily_forecasts.columns

Index(['date', 'min_temperature', 'max_temperature', 'total_precipitation',
       'avg_wind_speed', 'max_wind_speed', 'avg_relative_humidity',
       'avg_pressure', 'avg_cloud_fraction', 'dominant_weather_symbol'],
      dtype='object')